<a href="https://colab.research.google.com/github/xAIdrian/MachineLearning/blob/develop/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install pandas
!pip install jsonlines

In [ ]:
import pandas as pd
import jsonlines
import openai
import gspread
import time
import os

In [ ]:
openai.api_key = ''
training_filename = './output.jsonl'

In [ ]:
# Read the CSV file into a Pandas DataFrame
df = pd.read_csv('Copy of 100 content - Sheet1.csv')

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Assign the first 3 columns as string variables
    system_message = str(row[0])
    prompt_text = str(row[1])
    ideal_generated_text = str(row[2])

    # You can use col1, col2, and col3 as string variables in further processing
    # print(f"Row {index + 1}: {system}, {user}, {assistant}")
    # Format and save data to jsonl
    data = {
        "messages": [
            {"role": "system", "content": system_message},
            {"role": "user", "content": prompt_text},
            {"role": "assistant", "content": ideal_generated_text}
        ]
    }
    with jsonlines.open(training_filename, mode='a') as writer:
        writer.write(data)


In [ ]:
training_response = openai.File.create(
    file=open(training_filename, 'rb'), purpose='fine-tune'
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(training_filename, 'rb'), purpose='fine-tune'
)
validation_file_id = validation_response["id"]

print(f"Training file id: {training_file_id}")
print(f"Validation file id: {validation_file_id}")

In [ ]:
suffix_name = 'AdoBot'

response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name
)
response

In [ ]:
job_id = response['id']
response = openai.FineTuningJob.retrieve(job_id)
response

In [ ]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)
events = response['data']
events.reverse()

for event in events:
  print(event['message'])

In [ ]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response['fine_tuned_model']
response

In [ ]:
response = openai.ChatCompletion.create(
    model="<FINE TUNING JOB ID>",
    messages=arr,
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)
completion = response['choices'][0]['message']['content']
completion